<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/AI_Forced_Alignment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intel-tensorflow is required to run on CPU
!pip install torch torchaudio soundfile omegaconf intel-tensorflow tensorflow_hub

     |████████████████████████████████| 7.6MB 4.5MB/s 
     |████████████████████████████████| 386.6MB 43kB/s 
     |████████████████████████████████| 645kB 49.3MB/s 
     |████████████████████████████████| 460kB 44.7MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: torchaudio 0.7.2 has requirement torch==1.7.1, but you'll have torch 1.7.0+cu101 which is incompatible.
ERROR: intel-tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [3]:
import os
import torch
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

language = 'en'

# load provided utils using torch.hub for brevity
_, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

# see available models
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

# load the actual tf model
tf_model = tf_hub.load(models.stt_models.get(language).latest.tf)

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file(models.stt_models.get(language).latest.meta.sample, dst ='speech_orig.wav', progress=True)
test_files = ['speech_orig.wav']
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [4]:
tf_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f1bb9af1b00>

In [5]:
res

<tf.Tensor: shape=(1, 136, 999), dtype=float32, numpy=
array([[[9.9562091e-01, 2.1983124e-06, 4.2956504e-03, ...,
         7.4657960e-06, 6.9315120e-06, 1.2493843e-05],
        [2.3445174e-02, 1.3809526e-05, 9.7602135e-01, ...,
         3.1058439e-06, 1.8454979e-06, 3.6289515e-05],
        [2.6382620e-02, 6.0946192e-08, 8.4972955e-05, ...,
         1.2406194e-05, 1.3228365e-06, 9.7339523e-01],
        ...,
        [9.9961817e-01, 1.4285628e-06, 9.0514250e-06, ...,
         6.1206447e-05, 6.5136360e-06, 2.0248137e-05],
        [9.9961817e-01, 1.4285029e-06, 9.0513904e-06, ...,
         6.1204344e-05, 6.5132817e-06, 2.0246360e-05],
        [9.9961817e-01, 1.4285737e-06, 9.0529093e-06, ...,
         6.1196988e-05, 6.5128656e-06, 2.0244024e-05]]], dtype=float32)>

In [6]:
decoder

In [19]:
# construct a shape[1,136,999] tensor where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 0
the_tensor = []
for i in range(136):
  if i == value:
    the_tensor.append(list(np.ones(999)))
  else:
    the_tensor.append(list(np.zeros(999)))

the_tensor = np.array(the_tensor)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)


tf.Tensor(
[[[1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)


In [20]:
#res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(the_tensor.numpy())[0]))

# Test that the 999 does refer to time, and thus halving it will shorten the output
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))

In [43]:
res1, res2 = tf.split(res, [500,499],2)
print(res1.shape)
#print(decoder(torch.Tensor(res1.numpy())[0]))
'''---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-617167afc803> in <module>()
      1 res1, res2 = tf.split(res, [500,499],2)
      2 print(res1.shape)
----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))

1 frames
/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)
     62     def process(self,
     63                 probs, wav_len, word_align):
---> 64         assert len(self.labels) == probs.shape[1]
     65         for_string = []
     66         argm = torch.argmax(probs, axis=1)

AssertionError: '''

(1, 136, 500)


'---------------------------------------------------------------------------\nAssertionError                            Traceback (most recent call last)\n<ipython-input-38-617167afc803> in <module>()\n      1 res1, res2 = tf.split(res, [500,499],2)\n      2 print(res1.shape)\n----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))\n\n1 frames\n/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)\n     62     def process(self,\n     63                 probs, wav_len, word_align):\n---> 64         assert len(self.labels) == probs.shape[1]\n     65         for_string = []\n     66         argm = torch.argmax(probs, axis=1)\n\nAssertionError: '

In [53]:
# construct a shape[1,136,999] tensore where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 11
the_tensor = []
for i in range(999):
  if i == value:
    temp = list(np.ones(136))
    temp[10] = 0
    the_tensor.append(temp)
  else:
    temp = list(np.zeros(136))
    the_tensor.append(temp)
    if i == 12:
      temp[10] = 1


the_tensor = np.array(the_tensor)
the_tensor = np.swapaxes(the_tensor, 0, 1)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)

print(decoder(torch.Tensor(the_tensor.numpy())[0]))

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)
entoen


In [9]:
#@title Install dependencies

!pip install -q omegaconf
!pip install -q torchaudio
!pip install -q soundfile
!pip install -q pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from utils import (init_jit_model, 
                   split_into_batches,
                   read_audio,
                   read_batch,
                   prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


/content/silero-models/silero-models/silero-models


In [12]:
sample_rate = 16000

clear_output()
audio = record_audio(10)
wavfile.write('recorded.wav', sample_rate, (32767*audio).numpy().astype(np.int16))


#button = widgets.Button(description="Record Speech")
#button.on_click(_record_audio)
#display(button)

display(Audio(audio, rate=sample_rate, autoplay=True))
wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [41]:
len(decoder.labels)

999

In [42]:
decoder.blank_idx

0